<a href="https://colab.research.google.com/github/Devdeep-J-S/Vision-Transformers-CMS/blob/main/Vision_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name : Devdeep Shetranjiwala <br>
Email ID : devdeep0702@gmail.com 

## Specific Task : Vision Transformers

Description:
> Train a Transformer model of your choice on the dataset below to achieve the performance closest to your CNN model’s performance in Task 1. </br>
> Discuss the resulting performance of the 2 chosen architectures.
## (The discussion is done at the end of the code)

Datasets (Same as in Task 1): </br>
https://cernbox.cern.ch/index.php/s/AtBT8y4MiQYFcgc (Photons) </br>
https://cernbox.cern.ch/index.php/s/FbXw3V4XNyYB3oA (Electrons)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import h5py
from sklearn.model_selection import train_test_split

In [ ]:
print("Num of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num of GPUs Available:  1


In [ ]:
import requests
url = 'https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
r = requests.get(url, allow_redirects=True)
open('photons.hdf5', 'wb').write(r.content)
url = 'https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'
r = requests.get(url, allow_redirects=True)
open('electrons.hdf5', 'wb').write(r.content)

128927319

In [ ]:
file_electron = "electrons.hdf5"
file_photon = "photons.hdf5"

with h5py.File(file_electron, "r") as f1:
    X_elec = np.array(f1['X'][:])
    y_elec = np.array(f1['y'][:])
with h5py.File(file_photon, "r") as f2:
    X_phot = np.array(f2['X'][:])
    y_phot = np.array(f2['y'][:])
print(X_elec.shape)
print(X_phot.shape)
del(file_electron,file_photon)

(249000, 32, 32, 2)
(249000, 32, 32, 2)


In [ ]:
num_classes = 2
input_shape = (32, 32, 2)
X = np.append(X_elec, X_phot, axis=0)
y = np.append(y_elec, y_phot)
X.shape
del(X_elec,X_phot,y_elec,y_phot)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
del(X,y)

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes=2)
y_test = keras.utils.to_categorical(y_test, num_classes=2)

In [ ]:
learning_rate = 0.0001
batch_size = 256
num_epochs = 50
patch_size = 2
num_patches = (input_shape[0]//patch_size)**2
projection_dim = 64
num_heads = 2
transformer_units = [
    projection_dim * 2,
    projection_dim,
]
transformer_layers = 2
mlp_head_units = [512, 256]

In [ ]:
def mlp(x, hidden_units):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    patches = Patches(patch_size)(inputs)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0
        )(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, hidden_units=transformer_units)
        encoded_patches = layers.Add()([x3, x2])
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    features = mlp(representation, hidden_units=mlp_head_units)
    outputs = layers.Dense(num_classes, activation='softmax')(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def run_experiment(model):
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC()])
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.2,
                              patience=10, min_lr=1e-10   , verbose=1)
    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[reduce_lr]
    )
    return model, history

In [ ]:
vit_classifier = create_vit_classifier()
model, history = run_experiment(vit_classifier)

Epoch 1/50
1401/1401 [==============================] - 172s 113ms/step - loss: 0.6733 - auc: 0.6385 - val_loss: 0.6478 - val_auc: 0.6729 - lr: 1.0000e-04
Epoch 2/50
1401/1401 [==============================] - 163s 116ms/step - loss: 0.6318 - auc: 0.6976 - val_loss: 0.6121 - val_auc: 0.7287 - lr: 1.0000e-04
Epoch 3/50
1401/1401 [==============================] - 163s 116ms/step - loss: 0.6153 - auc: 0.7218 - val_loss: 0.6269 - val_auc: 0.7157 - lr: 1.0000e-04
Epoch 4/50
1401/1401 [==============================] - 163s 116ms/step - loss: 0.6047 - auc: 0.7355 - val_loss: 0.6086 - val_auc: 0.7398 - lr: 1.0000e-04
Epoch 5/50
1401/1401 [==============================] - 163s 116ms/step - loss: 0.5992 - auc: 0.7421 - val_loss: 0.5999 - val_auc: 0.7424 - lr: 1.0000e-04
Epoch 6/50
1401/1401 [==============================] - 163s 116ms/step - loss: 0.5926 - auc: 0.7495 - val_loss: 0.5970 - val_auc: 0.7497 - lr: 1.0000e-04
Epoch 7/50
1401/1401 [==============================] - 163s 116ms/ste

In [ ]:
model.evaluate(x_test,y_test)

3113/3113 [==============================] - 30s 10ms/step - loss: 0.5670 - auc: 0.7786


[0.5669955611228943, 0.7785612940788269]

Discuss the resulting performance of the 2 chosen architectures.
> The AUC score is low because of lower no. of epochs still it is comperable to keras and pytorch score of 0.8.